In [ ]:
from transformers import AutoModelForSequenceClassification  # 同样需要导入这个
from transformers import AutoTokenizer, DebertaV2Model, DebertaV2Config
from transformers import TrainingArguments
from transformers.models.deberta_v2.modeling_deberta_v2 import DebertaV2Encoder
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import Trainer

## 教师模型微调

In [ ]:
# 1. 定义要使用的预训练模型ID
model_id = 'microsoft/deberta-v3-base' # 指定了强大的 DeBERTa V3 Base 模型
# 2. 加载对应的分词器 (Tokenizer)
tokenizer = AutoTokenizer.from_pretrained(model_id)
# 3. 加载预训练模型用于序列分类任务
# AutoModelForSequenceClassification 会自动加载适合序列分类的模型结构
# 并在 DeBERTa 结构上添加一个分类头 (classification head)
model = AutoModelForSequenceClassification.from_pretrained(model_id)

In [ ]:
model.save_pretrained('deberta-v3-base-finetuned')
tokenizer.save_pretrained('deberta-v3-base-finetuned') # 分词器也一起保存，方便后续使用

## 创建并初始化学生模型

现在我们有了教师模型 (deberta-v3-base-finetuned)，接下来要创建一个更小的学生模型，并用教师模型的权重来初始化它。

1. 定义创建学生模型的函数 create_student

In [ ]:
# 定义权重拷贝函数
def copy_deberta_weights(teacher, student):
    #权重拷贝逻辑，见下方
    if isinstance(teacher, DebertaV2Model) or type(teacher).__name__.startswith('DebertaV2For'): # 检查是否是DeBERTa模型或其变体 (如序列分类模型)
        # 递归地对模型的直接子模块进行拷贝
        for teacher_part, student_part in zip(teacher.children(), student.children()):
            copy_deberta_weights(teacher_part, student_part) # 对子模块递归调用

    elif isinstance(teacher, DebertaV2Encoder): # 如果当前部分是 DeBERTa 的编码器
        # 提取教师模型的编码层列表
        teacher_encoding_layers = [layer for layer in next(teacher.children())] # next(teacher.children()) 获取编码器内部的层模块
        # 提取学生模型的编码层列表
        student_encoding_layers = [layer for layer in next(student.children())]
        # 关键：拷贝部分层权重
        # 学生模型的第 i 层，拷贝教师模型的第 2*i 层权重
        for i in range(len(student_encoding_layers)):
            student_encoding_layers[i].load_state_dict(teacher_encoding_layers[2*i].state_dict())

    else: # 对于其他非特定处理的模块 (如 Embedding 层, 分类头等)
        # 直接加载教师模型的权重字典到学生模型的对应模块
        student.load_state_dict(teacher.state_dict())

*   它首先检查传入的 `teacher` 和 `student` 部分是否是 DeBERTa 的整体模型或其变体。如果是，它就递归地对模型的子模块（如 embeddings, encoder）调用自身，确保深入到每一部分。
*   当递归到 `DebertaV2Encoder`（编码器）部分时，它执行关键的层拷贝操作：
    *   它获取教师和学生编码器内部的所有 Transformer 层。
    *   然后，它遍历学生模型的每一层 (`i`)，将教师模型的**偶数层** (`2*i`) 的权重 (`state_dict`) 加载到学生模型的对应层中。例如，学生第 0 层加载教师第 0 层，学生第 1 层加载教师第 2 层，以此类推。这就是“使用教师模型的某些层次”的实现。
*   对于模型的其他部分（比如词嵌入层 Embedding、最后的分类层 Classifier 等），它采用直接拷贝的方式，即将教师模型对应部分的 `state_dict` 直接加载到学生模型的对应部分。


补充：在 Hugging Face 的 Transformers 库中，DebertaV2Model 是 基础模型，而 DebertaV2For... 是一些 在基础模型上加了任务头（head） 的模型。

`teacher.children()` 和 `student.children()` 是 PyTorch 中 `nn.Module` 类（几乎所有的模型和层都继承自它）提供的方法。

它的作用是**返回一个迭代器 (iterator)，这个迭代器包含了该模块的直接子模块 (direct child modules)**。

**举个例子：**

假设你的 `teacher_model` (它是一个 `nn.Module`) 的结构大致如下：

```
DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(...)
    (encoder): DebertaV2Encoder(...)
    ...
  )
  (pooler): ContextPooler(...)
  (classifier): Linear(...)
  (dropout): StableDropout(...)
)
```

当你调用 `teacher_model.children()` 时，它会返回一个迭代器，依次产生：

1.  `DebertaV2Model` 对象 (名为 `deberta`)
2.  `ContextPooler` 对象 (名为 `pooler`)
3.  `Linear` 对象 (名为 `classifier`)
4.  `StableDropout` 对象 (名为 `dropout`)

**它只会返回直接的子模块。** 它不会返回孙子模块。例如，它不会直接返回 `DebertaV2Encoder` 里面的 Transformer 层，因为 `DebertaV2Encoder` 本身是 `DebertaV2Model` 的子模块，而不是 `DebertaV2ForSequenceClassification` 的直接子模块。


In [ ]:
# 定义创建学生模型的函数
def create_student(teacher_model):
    # a. 获取教师模型的配置，并转换为字典
    configuration = teacher_model.config.to_dict()

    # b. 修改配置：将隐藏层数量减半 (// 是整数除法)
    original_num_layers = configuration["num_hidden_layers"]
    configuration["num_hidden_layers"] //= 2
    print(f"Teacher layers: {original_num_layers}, Student layers: {configuration['num_hidden_layers']}") # 打印层数变化

    # c. 使用修改后的配置创建 DebertaV2Config 对象
    student_config = DebertaV2Config.from_dict(configuration)

    # d. 使用学生配置创建学生模型实例
    # type(teacher_model) 获取教师模型的类 (例如 AutoModelForSequenceClassification)
    # 然后用这个类和学生配置来创建学生模型，确保结构兼容（除了层数）
    student_model = type(teacher_model)(config=student_config) # 注意这里用 config= 传递配置

    # e. 调用权重拷贝函数，将教师的部分权重初始化给学生模型
    copy_deberta_weights(teacher_model, student_model)

    # f. 返回创建并初始化好的学生模型
    return student_model

你可以把它想象成：我们先拿到教师模型的设计蓝图 (`.config`)，然后把它复印一份变成我们能随意涂改的草图 (`.to_dict()`)，接着我们在这份草图上修改我们想要简化的部分（比如减少楼层数），最后拿着修改后的草图去盖一栋新楼（学生模型）。

初始化 Hugging Face 模型主要有两种方式：

1.  **使用类的构造函数 `__init__` 并传入 `config` 对象:**
    *   **代码示例:** `model = DebertaV2ForSequenceClassification(config=my_config)` 或我们例子中的 `student_model = type(teacher_model)(config=student_config)`
    *   **作用:** 根据你提供的 `config` 对象中定义的**架构参数**（如层数、隐藏单元数等）来**从零开始构建模型骨架**。
    *   **权重:** 这种方式创建的模型，其内部的**权重是随机初始化**的。它**不会**加载任何预训练好的权重。
    *   **何时使用:**
        *   当你想要训练一个**全新的模型**，完全不依赖预训练权重时。
        *   当你修改了模型的架构（比如改变层数），需要根据这个**新的架构**创建一个模型实例时（就像我们为学生模型所做的）。创建之后，你可能需要手动加载部分权重（如我们的 `copy_deberta_weights`）或开始全新的训练。

2.  **使用类方法 `from_pretrained()`:**
    *   **代码示例:** `model = AutoModelForSequenceClassification.from_pretrained('microsoft/deberta-v3-base')` 或 `teacher_model = AutoModelForSequenceClassification.from_pretrained('./deberta-v3-base-finetuned')`
    *   **作用:** 从一个指定的**来源**（Hugging Face Hub 上的模型名称或本地保存模型的路径）加载**模型架构**和**对应的预训练权重**。
    *   **权重:** 这是与第一种方式最关键的区别——`from_pretrained` 会自动**加载**与该模型关联的**已训练好的权重**（无论是初始的预训练权重还是微调后的权重）。
    *   **配置:** `from_pretrained` 会自动从来源加载 `config.json` 文件来确定模型架构。你也可以在调用 `from_pretrained` 时传入 `config` 参数来覆盖某些配置，但这不太常见，主要目的是加载权重。
    *   **何时使用:**
        *   当你想要利用强大的**预训练模型**作为起点进行微调时（这是最常见的用法）。
        *   当你需要加载之前**保存好**的、已经微调过的模型（比如我们加载教师模型）或训练过程中的检查点时。

**总结关键区别:**

| 特性         | `__init__(config=...)`                  | `from_pretrained(...)`                       |
| :----------- | :--------------------------------------- | :------------------------------------------- |
| **输入**     | 必须提供一个 `config` 对象             | 提供模型标识符 (str) 或本地路径 (str)        |
| **架构来源** | 完全由传入的 `config` 对象决定         | 从来源加载 `config.json` 文件决定          |
| **权重来源** | **随机初始化**                           | **从来源加载已保存的权重**                 |
| **主要目的** | 构建指定架构的模型骨架 (无预训练权重) | 加载完整的预训练或已保存模型 (带权重)      |

在我们的知识蒸馏场景中：

*   我们用 `from_pretrained` 加载**教师模型**，因为它需要包含微调后的知识（权重）。
*   我们用 `type(teacher_model)(config=student_config)`（本质是 `__init__`）来创建**学生模型**的骨架，因为它需要一个**新的、层数减少的架构**，权重是随后通过 `copy_deberta_weights` 部分初始化的，而不是直接加载某个预训练版本。

## 调用函数并保存学生模型

In [ ]:
# a. 加载之前保存的微调好的教师模型
teacher_model_path = 'deberta-v3-base-finetuned'
teacher_model = AutoModelForSequenceClassification.from_pretrained(teacher_model_path)

# b. 调用 create_student 函数创建学生模型
student_model = create_student(teacher_model)

# c. 定义学生模型保存路径
student_model_path = 'deberta-v3-base-student'

# d. 保存初始化后的学生模型 (只保存模型本身，分词器后面会用教师的)
student_model.save_pretrained(student_model_path)

# (可选) 打印学生模型结构，确认层数是否减少
print(student_model)

## 设置蒸馏训练

定义蒸馏训练参数类 DistillationTrainingArguments。
这个类继承自 Hugging Face 的 TrainingArguments，并添加了 alpha 和 temperature 两个额外的参数。

In [ ]:
# 定义一个类，继承自 TrainingArguments，用于存放蒸馏相关的超参数
class DistillationTrainingArguments(TrainingArguments):
    def __init__(self, *args, alpha=0.5, temperature=2.0, **kwargs):
        # 调用父类的构造函数，传递常规的训练参数
        super().__init__(*args, **kwargs)

        # 初始化蒸馏特定的参数
        self.alpha = alpha           # alpha 控制硬标签损失 (student_loss) 的权重
        # (1 - alpha) 控制软标签损失 (distillation_loss) 的权重
        self.temperature = temperature # temperature 用于计算软标签和软预测时的温度

定义自定义蒸馏训练器 DistillationTrainer。这个类继承自 Hugging Face 的 Trainer，并重写了核心的 compute_loss 方法来实现蒸馏逻辑。

In [ ]:
# 定义一个类，继承自 Trainer，用于执行蒸馏训练
class DistillationTrainer(Trainer):
    def __init__(self, *args, teacher_model=None, **kwargs):
        # 调用父类的构造函数，传递常规参数 (包括 student_model 会被传给父类的 model 参数)
        super().__init__(*args, **kwargs)
        # 保存教师模型
        self.teacher = teacher_model
        # 确保教师模型和学生模型在同一个设备上 (CPU 或 GPU)
        # self.model 是父类 Trainer 保存的学生模型
        self._move_model_to_device(self.teacher, self.model.device)
        # 将教师模型设置为评估模式，因为教师模型在蒸馏过程中不参与训练，只提供输出
        self.teacher.eval()

    # 重写 compute_loss 方法，这是蒸馏的核心
    def compute_loss(self, model, inputs, return_outputs=False):
        """
        如何计算损失：
        1. 计算学生模型的输出和标准损失 (硬损失)。
        2. 在不计算梯度的情况下，计算教师模型的输出。
        3. 使用 KL 散度计算学生软预测和教师软标签之间的蒸馏损失。
        4. 使用 alpha 加权组合硬损失和蒸馏损失。
        """
        # `model` 在这里就是学生模型 (因为它是传递给父类 Trainer 的 model)
        # 1. 计算学生模型的输出
        # **inputs 将输入的字典解包传递给模型
        outputs_student = model(**inputs)

        # 提取学生模型的原始损失 (通常是交叉熵损失，针对硬标签)
        student_loss = outputs_student.loss
        # 提取学生模型的 logits (softmax 前的原始输出)
        logits_student = outputs_student.logits

        # 2. 计算教师模型的输出 (在 no_grad 上下文中，不计算梯度)
        with torch.no_grad():
            outputs_teacher = self.teacher(**inputs)
        # 提取教师模型的 logits
        logits_teacher = outputs_teacher.logits

        # 3. 计算蒸馏损失 (KL 散度)
        #    a. 从训练参数中获取温度 T 和 alpha
        temperature = self.args.temperature
        alpha = self.args.alpha

        #    b. 定义 KL 散度损失函数，reduction='batchmean' 表示对 batch 内样本的损失求平均
        loss_fct = nn.KLDivLoss(reduction="batchmean")

        #    c. 计算蒸馏损失
        #       F.log_softmax(logits_student / temperature, dim=-1): 学生 logits 除以温度 T，然后取 log_softmax
        #       F.softmax(logits_teacher / temperature, dim=-1):   教师 logits 除以温度 T，然后取 softmax (作为目标分布)
        #       KLDivLoss 需要输入是 log 概率，目标是普通概率
        distillation_loss = loss_fct(
            F.log_softmax(logits_student / temperature, dim=-1),
            F.softmax(logits_teacher / temperature, dim=-1)
        )

        #    d. 乘以温度的平方进行缩放 (Hinton 论文中的做法)
        distillation_loss = distillation_loss * (temperature ** 2)

        # 4. 组合损失
        #    最终损失 = alpha * 硬损失 + (1 - alpha) * 蒸馏损失
        loss = alpha * student_loss + (1.0 - alpha) * distillation_loss

        # 根据 return_outputs 参数决定返回值
        # 如果为 True，返回损失和学生模型的输出；否则只返回损失
        return (loss, outputs_student) if return_outputs else loss

*   **背景：** 当我们计算损失时，输入通常是一个**批次 (batch)** 的数据。例如，`logits_student` 和 `logits_teacher` 的形状可能是 `[batch_size, num_classes]`。计算 KL 散度会得到一个针对**每个样本**的损失值，所以我们会得到一个形状类似 `[batch_size]` 的张量，其中每个元素是对应样本的 KL 散度损失。
*   **聚合 (Reduction)：** 为了进行反向传播和模型优化，我们通常需要将这一个批次的**所有样本的损失**聚合成一个**单一的标量值 (scalar)**。`reduction` 参数就是用来控制这个聚合方式的。
*   **`"batchmean"` 的具体含义 (对于 `KLDivLoss`)：**
    *   它指示 PyTorch 首先计算每个样本的 KL 散度损失。
    *   然后，将这些损失值在**批次维度 (batch dimension) 上求平均值**。
    *   (根据 `KLDivLoss` 的文档，它实际上是先在特征维度上求和，然后在批次维度上求平均。但最终效果是得到一个代表整个批次平均损失的标量值)。
    *   **简单理解：** 计算出批次中每个样本的模仿损失后，取它们的平均值作为这个批次的最终模仿损失。

`KLDivLoss` (以及许多其他 PyTorch 损失函数) 通常支持以下几种 `reduction` 模式：

*   **`'mean'`:** 计算损失的平均值。对于 `KLDivLoss`，`'mean'` 和 `'batchmean'` 的效果通常是一样的（都是求批次的平均损失）。这是最常用的选项，因为它使得损失的大小不直接受批次大小的影响。
*   **`'sum'`:** 计算损失的总和。即把批次中所有样本的损失加起来。如果使用这个，损失值会随着批次增大而增大，可能需要相应调整学习率。
*   **`'none'`:** 不进行聚合。损失函数将返回一个与输入批次大小相同的张量，其中包含每个样本的损失值。这在需要对每个样本的损失进行单独处理时有用，但在标准训练循环中不直接用于反向传播（因为反向传播需要标量损失）。

 `KLDivLoss` 通常期望的输入 (Input) 是对数概率 (log-probabilities)，而目标 (Target) 是普通概率 (probabilities)。这就是为什么在 `compute_loss` 函数中，我们对学生 logits 使用 `F.log_softmax`，而对教师 logits 使用 `F.softmax`。

## 加载模型、数据并进行蒸馏训练

In [ ]:
# 1. 定义模型路径
student_id = "deberta-v3-base-student" # 之前保存的学生模型路径
teacher_id = "deberta-v3-base-finetuned" # 之前保存的教师模型路径

# 2. 加载教师模型
teacher_model = AutoModelForSequenceClassification.from_pretrained(teacher_id)
# 3. 加载初始化好的学生模型
student_model = AutoModelForSequenceClassification.from_pretrained(student_id)

# 4. 加载分词器 (用教师的或学生的都可以，因为它们应该是一样的)
tokenizer = AutoTokenizer.from_pretrained(teacher_id) # 或者 student_id

In [ ]:
# 5. 配置蒸馏训练参数
output_dir = "checkpoint" # 训练过程中保存检查点的目录
logging_dir = './logs'     # 保存 TensorBoard 日志的目录

training_args = DistillationTrainingArguments(
    output_dir=output_dir,               # 输出目录
    num_train_epochs=3,                  # 训练轮数
    per_device_train_batch_size=2,       # 每个 GPU/CPU 上的训练批次大小
    per_device_eval_batch_size=4,        # 每个 GPU/CPU 上的评估批次大小
    gradient_accumulation_steps=8,       # 梯度累积步数 (增大有效批次大小)
    gradient_checkpointing=True,         # 使用梯度检查点节省显存 (会稍慢)
    warmup_steps=500,                    # 学习率预热步数
    weight_decay=0.01,                   # 权重衰减系数 (L2 正则化)
    logging_dir=logging_dir,             # 日志目录
    logging_steps=100,                   # 每隔多少步记录一次日志
    evaluation_strategy="epoch",         # 每个 epoch 结束时进行一次评估
    save_strategy="no",                  # 这里设置为不自动保存模型 (可以改为 "epoch" 按轮保存)
    # --- 蒸馏特定参数 ---
    alpha=0.5,                           # 硬损失和软损失的权重平衡因子
    temperature=4.0                      # 软化标签的温度 (文档示例用了 4.0)
)

In [ ]:
# 6. 创建 DistillationTrainer 实例
trainer = DistillationTrainer(
    model=student_model,                 # 学生模型传递给 'model' 参数
    teacher_model=teacher_model,         # 教师模型传递给自定义的 'teacher_model' 参数
    args=training_args,                  # 传入包含蒸馏参数的训练配置
    train_dataset=tokenized_train,       # 训练数据集
    eval_dataset=tokenized_val,          # 验证数据集
    data_collator=data_collator,         # 数据整理器
    tokenizer=tokenizer,                 # 分词器 (用于填充等)
    compute_metrics=compute_metrics      # 评估指标计算函数
)

In [ ]:
# 7. 开始蒸馏训练
print("Starting distillation training...")
trainer.train()
print("Training finished.")

In [ ]:
# (可选) 保存最终训练好的学生模型
final_student_model_path = "deberta-v3-base-student-distilled"
trainer.save_model(final_student_model_path)
print(f"Final distilled student model saved to {final_student_model_path}")

In [ ]:
import numpy as np
import pandas as pd
import logging

# --- 假设 tokenized_test 和 test 数据集已准备好 ---
# tokenized_test: 经过分词器处理的测试数据集 (Dataset 对象)
# test: 原始测试数据集，包含 'id' 等信息，用于保存结果 (例如 Pandas DataFrame 或 Dataset 对象)

# (示例：假设 test 是加载的原始数据集的一部分)
# test = dataset["test"].shard(num_shards=2, index=1) # 对应之前 tokenized_test 的原始数据
# --- 数据准备结束 ---


# 1. 使用训练好的 trainer 在测试集上进行预测
# trainer 内部持有训练完成的学生模型
print("Evaluating the distilled student model on the test set...")
prediction_outputs = trainer.predict(tokenized_test)

# prediction_outputs 是一个 PredictionOutput 对象 (或类似结构)
# prediction_outputs.predictions 包含了模型输出的 logits (通常是 prediction_outputs[0])
# prediction_outputs.label_ids 包含了真实的标签 (通常是 prediction_outputs[1]，如果测试集有标签)
# prediction_outputs.metrics 包含了计算得到的评估指标 (如果 compute_metrics 函数被调用)

print("Prediction metrics:", prediction_outputs.metrics) # 打印评估指标 (如准确率)

# 2. 从预测输出中提取最终的预测标签
# 获取模型的原始输出 logits (通常在第一个位置)
logits = prediction_outputs.predictions # 或者 prediction_outputs[0]
# 找到每个样本概率最高的类别索引作为预测标签
test_pred = np.argmax(logits, axis=-1).flatten() # argmax 找最大值索引，flatten 转为一维数组

# 3. (可选) 打印部分预测结果，直观感受
print("Sample predictions (first 20):", test_pred[:20])
# 如果测试集有真实标签，可以比较一下
# if prediction_outputs.label_ids is not None:
#     true_labels = prediction_outputs.label_ids.flatten()
#     print("Sample true labels (first 20):", true_labels[:20])

# 4. 将预测结果格式化并保存到文件
# 创建一个 Pandas DataFrame 来存储结果
# 假设原始 'test' 数据集里有 'id' 这一列
# 如果 'test' 是 Dataset 对象，可能需要先转换为 Pandas: test_df = test.to_pandas()
# 这里假设 test 是可以直接按索引访问 id 的结构，例如 test['id']
# 请根据你的实际 test 数据结构调整 data={"id": ...} 部分
try:
    # 尝试假设 test 是字典或类似结构
    result_data = {"id": test["id"], "sentiment": test_pred}
except TypeError:
    # 如果 test 是 Dataset, 可能需要迭代获取 id
    test_ids = [item['id'] for item in test] # 示例，具体字段名可能不同
    result_data = {"id": test_ids, "sentiment": test_pred}

result_output = pd.DataFrame(data=result_data)

# 定义保存路径和文件名
output_csv_path = "./result/deberta_base_student_distilled.csv" # 修改了文件名以区分
# 将 DataFrame 保存为 CSV 文件
# index=False 表示不将 DataFrame 的索引写入文件
# quoting=3 (QUOTE_NONNUMERIC) 表示给非数字加上引号，符合某些提交格式要求
result_output.to_csv(output_csv_path, index=False, quoting=3)

logging.info(f'Result saved to {output_csv_path}!')
print(f"Predictions saved to {output_csv_path}")

# 5. 分析结果 (手动步骤)
# - 查看保存的 CSV 文件。
# - 将 prediction_outputs.metrics 中的指标与教师模型在该测试集上的指标进行比较。
# - 分析学生模型在哪些样本上表现好/差。
# - 根据文档建议，可以尝试不同的学生模型、alpha、temperature 值，重复训练和评估过程，找到最佳组合。